In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import networkx as nx


In [2]:
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display_html
from IPython.display import display

In [5]:
df_pib = pd.read_excel('Base Total.xlsx',engine='openpyxl',sheet_name='Final')

## Histograma

In [6]:
sheets = [
    'BaseTotal','Europa',
    'America','Asia','Oceana',
    'Africa','SICA','CARICOM',
    'ASEAN','European Union',
    'MCCA','NAFTA','ALADI',
    'MERCOSUR','PACIFIC ALLIANCE',
    'CAN'
]

In [7]:
options = ['Export', 'Import','Tij']

In [10]:
df_dictionary = {key:pd.read_excel('trade.base.xlsx',engine='openpyxl',sheet_name=key) for key in sheets}

In [11]:
df_dictionary['BaseTotal'].head()

,Year,Reporter,Partner,Export,Import,Tij,BOT,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1991,Ecuador,Albania,4747.0,5229.0,9976.0,-482.0,NaN,NaN,Etiquetas de fila,NaN
1,1992,Ecuador,Albania,1791.0,202016.0,203807.0,-200225.0,NaN,1.0,Afghanistan,Asia
2,1993,Ecuador,Albania,NaN,3299168.0,3299168.0,-3299168.0,NaN,2.0,Albania,Europa
3,1994,Ecuador,Albania,NaN,211021.0,211021.0,-211021.0,NaN,3.0,Algeria,Africa
4,1995,Ecuador,Albania,67234.0,11519.0,78753.0,55715.0,NaN,4.0,Angola,Africa


In [12]:
def get_df_from_top(df,columna,th):
  total_exports = [{'country':country,'total':df[df['Partner']==country][columna].sum()} for country in df['Partner'].unique()]
  df_exports = pd.DataFrame(total_exports)
  df_exports = df_exports.sort_values(by=['total'],ascending=False)
  df_top = df_exports.iloc[:th]
  return df_top

In [13]:
def grafico_hist(grupo:str,columna:str,limit:int):
  df = df_dictionary[grupo]
  df_plot = get_df_from_top(df,columna,limit)
  plt.figure(figsize=(15,6))
  plt.plot(df_plot['country'],df_plot['total'], linestyle='--', marker='o')

In [14]:
max_countries = len(df_dictionary['BaseTotal']['Partner'].unique())

In [15]:
max_countries

129

Explicación:

* Se puede seleccionar grupo de países (Todos los países por defecto)
* Cada punto representa el total de la variable Datos (Export por defecto) 
* Se cuenta con una barra para modificar el número de países que se muestran (10 por defecto)

In [16]:
X = widgets.Dropdown(options=sheets, value='BaseTotal',
    description='Grupo de países',
    layout=widgets.Layout(width='70%'))
X.style = {'description_width': '300px'}
Y = widgets.Dropdown(options=options, value='Export',
    description='Datos',
    layout=widgets.Layout(width='70%'))
Y.style = {'description_width': '300px'}
Z = widgets.SelectionSlider(options=range(0,max_countries,5), value=10,
    description='Top',
    layout=widgets.Layout(width='70%'))
Z.style = {'description_width': '300px'}

ui = widgets.VBox(children=[X,Y,Z])
out = widgets.interactive_output(grafico_hist, {'grupo': X,'columna':Y, 'limit':Z})
display(ui,out)

Output()

## Spring layout

In [118]:
def get_G_star(df_plot,BOT):
  all_countries = ['Ecuador']
  all_countries  += [country for country in df_plot['country']]
  total_exports = {country:total for country, total in zip(df_plot['country'], df_plot['total'])}
  G = nx.star_graph(all_countries)
  gdp_max = df_pib['gdp'].max()
  countries_gdp = {country:df_pib[df_pib['country']==country]['gdp'].sum()/gdp_max*100 for country in all_countries[1:]}
  countries_gdp['Ecuador']=100
  node_sizes = [countries_gdp[node] for node in G.nodes()]
  edges = G.edges()
  color_map = ['yellow']
  color_map += ['green' if BOT[country]>=0 else 'red' for country in all_countries[1:]]
  export_max = max(total_exports.values())
  width_list = [total_exports[edge[1]]/export_max*3 for edge in edges]
  width_dict = {edge:{'weight':total_exports[edge[1]]/export_max*2} for edge in edges}
  nx.set_edge_attributes(G, width_dict)
  pos=nx.spring_layout(G,weight='weight')
  plt.figure(figsize=(8,6))
  nx.draw(G,pos=pos,with_labels = True,width=width_list,node_color=color_map,node_size=node_sizes)
  
  plt.plot()


In [119]:
def spring_plot(grupo:str,columna:str,th:int):
  df = df_dictionary[grupo]
  df_plot = get_df_from_top(df,columna,th)
  countries = df_plot['country']
  total_BOT = {country:df[df['Partner']==country]['BOT'].sum() for country in countries}
  get_G_star(df_plot,total_BOT)

Explicación:

* Se puede seleccionar grupo de países (Todos los países por defecto)
* El tamaño de cada nodo representa el PIB
* La distancia entre el centro (Ecuador) y el nodo representa la variable seleccionada (Export por defecto)
* Se cuenta con una barra para modificar el número de países que se muestran (10 por defecto)

In [155]:
Z2 = widgets.SelectionSlider(options=range(1,max_countries+1,1), value=10,
    description='Top',
    layout=widgets.Layout(width='70%'))
Z2.style = {'description_width': '300px'}
ui = widgets.VBox(children=[X,Y,Z2])
out = widgets.interactive_output(spring_plot, {'grupo': X, 'columna':Y, 'th':Z2})
display(ui,out)

Output()

## Spectral Layout

Este gráfico utiliza una librería de agrupamiento, pero los resultados no son muy buenos.

In [121]:
def draw_spectral_G(df_plot,BOT):
  all_countries = ['Ecuador']
  all_countries  += [country for country in df_plot['country']]
  total_exports = {country:total for country, total in zip(df_plot['country'], df_plot['total'])}
  G = nx.star_graph(all_countries)
  edges = G.edges()
  color_map = ['yellow']
  color_map += ['green' if BOT[country]>=0 else 'red' for country in all_countries[1:]]
  export_max = max(total_exports.values())
  width_list = [total_exports[edge[1]]/export_max*3 for edge in edges]
  width_dict = {edge:{'weight':total_exports[edge[1]]/export_max} for edge in edges}
  nx.set_edge_attributes(G, width_dict)
  pos=nx.spectral_layout(G,weight='weight')
  plt.figure(figsize=(8,6))
  nx.draw(G,pos=pos,with_labels = True,width=width_list,node_color=color_map)
  plt.plot()


In [122]:
def spectral_plot(grupo:str,columna:str,th:int):
  df = df_dictionary[grupo]
  df_plot = get_df_from_top(df,columna,th)
  countries = df_plot['country']
  total_BOT = {country:df[df['Partner']==country]['BOT'].sum() for country in countries}
  draw_spectral_G(df_plot,total_BOT)

In [123]:
ui = widgets.VBox(children=[X,Y,Z2])
out = widgets.interactive_output(spectral_plot, {'grupo': X, 'columna':Y, 'th':Z2})
display(ui,out)

Output()

## Agrupamiento mediante una circunferencia

In [124]:
from scipy.spatial import distance

In [146]:
def get_G_circle_star(df_plot,BOT,k):
  all_countries = ['Ecuador']
  all_countries  += [country for country in df_plot['country']]
  total_exports = {country:total for country, total in zip(df_plot['country'], df_plot['total'])}
  G = nx.star_graph(all_countries)
  gdp_max = df_pib['gdp'].max()
  countries_gdp = {country:df_pib[df_pib['country']==country]['gdp'].sum()/gdp_max*100 for country in all_countries[1:]}
  countries_gdp['Ecuador']=100
  node_sizes = [countries_gdp[node] for node in G.nodes()]
  edges = G.edges()
  export_max = max(total_exports.values())
  width_list = [total_exports[edge[1]]/export_max*3 for edge in edges]
  width_dict = {edge:{'weight':total_exports[edge[1]]/export_max*2} for edge in edges}
  nx.set_edge_attributes(G, width_dict)
  pos=nx.spring_layout(G,weight='weight')
  center = pos['Ecuador']
  distances = get_distances_from_center(pos,center)
  radius_list = get_k_radius(k,max(distances.values()))
  
  figure, axes = plt.subplots()
  for radius in radius_list:
    Drawing_uncolored_circle = plt.Circle( center,
                                      radius,
                                      fill = False )
 
    axes.set_aspect( 1 )
    axes.add_artist( Drawing_uncolored_circle )
  axes.set_aspect( 1 )
  node_color = get_color_dict(radius_list,distances)
  color_map = [node_color[country] for country in G.nodes()]
  nx.draw(G,pos=pos,with_labels = True,width=width_list,node_color=color_map,node_size=300)
  figure.set_figheight(8)
  figure.set_figwidth(8)
  plt.plot()


In [147]:
def get_color_dict(radius_list,distances):
  colors = [i for i in range(len(radius_list))]
  color_map = {}
  for i in range(len(radius_list)-1,-1,-1):
    for country in distances.keys():
      current_distance = distances[country]
      if(current_distance<=radius_list[i]):
        color_map[country]=i
  return color_map

In [148]:
def get_distances_from_center(pos,center):
  distances = {}
  for country in pos.keys():
    distances[country] = distance.euclidean(pos[country],center)
  return distances

In [149]:
def get_k_radius(k,max_dist):
  radius1 = max_dist/k
  radius_list = [i for i in np.arange(radius1, max_dist, radius1)]
  return radius_list+[max_dist]

In [150]:
def spring_circular_plot(grupo:str,columna:str,th:int,k:int):
  df = df_dictionary[grupo]
  df_plot = get_df_from_top(df,columna,th)
  countries = df_plot['country']
  total_BOT = {country:df[df['Partner']==country]['BOT'].sum() for country in countries}
  get_G_circle_star(df_plot,total_BOT,k)

Explicación:

* Se puede seleccionar grupo de países (Todos los países por defecto)
* La distancia entre el centro (Ecuador) y el nodo representa la variable seleccionada (Export por defecto)
* Se cuenta con una barra para modificar el número de países que se muestran (10 por defecto)
* Se cuenta con una barra para seleccionar el número de circunferencias (3 por defecto). Esto permite agrupar países de acuerdo a la distancia.

In [154]:
X = widgets.Dropdown(options=sheets, value='BaseTotal',
    description='Grupo de países',
    layout=widgets.Layout(width='70%'))
X.style = {'description_width': '300px'}
Y = widgets.Dropdown(options=options, value='Export',
    description='Datos',
    layout=widgets.Layout(width='70%'))
Y.style = {'description_width': '300px'}
Z2 = widgets.SelectionSlider(options=range(0,max_countries+1,1), value=10,
    description='Top',
    layout=widgets.Layout(width='70%'))
Z2.style = {'description_width': '300px'}
K = widgets.SelectionSlider(options=range(1,11,1), value=3,
    description='Círculos',
    layout=widgets.Layout(width='70%'))
K.style = {'description_width': '300px'}
ui = widgets.VBox(children=[X,Y,Z2,K])
out = widgets.interactive_output(spring_circular_plot, {'grupo': X, 'columna':Y, 'th':Z2, 'k':K})
display(ui,out)

Output()